# CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
import numpy as np

from pymoo.algorithms.soo.convex.nonderiv.cmaes import CMAES
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

# algorithm = SimpleCMAES(normalize=True)
algorithm = CMAES(x0=np.random.random(problem.n_var))
# algorithm = BIPOPCMAES(restarts=4)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=True)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")


n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  1.158539155 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.997658251 |  0.096291243 |  0.09402 |  0.10254 |  1.18836
    3 |      21 |  0.936751712 |  0.090650246 |  0.08566 |  0.09458 |  1.24141
    4 |      31 |  0.775909950 |  0.090893687 |  0.08351 |  0.09609 |  1.34866
    5 |      41 |  0.727193990 |  0.091777904 |  0.08345 |  0.09717 |  1.37858
    6 |      51 |  0.571437494 |  0.095692866 |  0.08817 |  0.10204 |  1.49688
    7 |      61 |  0.461683428 |  0.103287850 |  0.09372 |  0.11405 |  1.63225
    8 |      71 |  0.348470500 |  0.111394186 |  0.09648 |  0.12775 |  1.85610
    9 |      81 |  0.204324605 |  0.131427837 |  0.11446 |  0.15418 |  1.93216
   10 |      91 |  0.204324605 |  0.143737429 |  0.12222 |  0.16868 |  2.02232
   11 |     101 |  0.184715565 |  0.143149117 |  0.12082 |  0.16612 |  2.03758
   12 |     111 |  0.184715565 |  0.162196017 |  0.1

CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  1.158539155 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  1.013845231 |  0.089330433 |  0.08610 |  0.09012 |  1.12611
    3 |      21 |  0.940747932 |  0.089154982 |  0.08422 |  0.09236 |  1.23746
    4 |      31 |  0.819625261 |  0.094465098 |  0.08862 |  0.09940 |  1.33470
    5 |      41 |  0.710532787 |  0.100553010 |  0.09335 |  0.10502 |  1.40779
    6 |      51 |  0.529377949 |  0.119889775 |  0.11314 |  0.12644 |  1.52814
    7 |      61 |  0.529377949 |  0.137270328 |  0.12650 |  0.14696 |  1.49951
    8 |      71 |  0.499407781 |  0.160063763 |  0.14773 |  0.17628 |  1.54963
    9 |      81 |  0.257501124 |  0.163557931 |  0.15205 |  0.17967 |  1.58326
   10 |      91 |  0.257501124 |  0.175328316 |  0.16210 |  0.18905 |  1.62561
Best solution found: 
X = [0.47462481 0.58825433 0.47116682 0.86216429 0.57338371 0.77044172
 0.51605983 0.59291783 0.51282259 0.32

In [3]:
res = minimize(problem,
               algorithm,
               ('n_evals', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  1.158539155 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.970643998 |  0.101053017 |  0.09653 |  0.10513 |  1.31107
    3 |      21 |  0.964969389 |  0.106784041 |  0.10114 |  0.11635 |  1.45907
    4 |      31 |  0.767483684 |  0.117649535 |  0.10971 |  0.12643 |  1.63011
    5 |      41 |  0.626366061 |  0.128005359 |  0.11749 |  0.13999 |  1.69529
    6 |      51 |  0.360025265 |  0.141847939 |  0.12269 |  0.16266 |  1.84158
Best solution found: 
X = [0.72632083 0.54067538 0.52129209 0.71759743 0.57792993 0.87725933
 0.79076126 0.53648513 0.40262363 0.37512182]
F = [0.36002527]


Also, easily restarts can be used, which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-9.94958639e-01  6.36055653e-10]
F = [0.99495906]


Our framework internally calls the `cma.fmin2` function. All parameters that can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-4.50144117e-07 -9.49562041e-07]
F = [2.19085194e-10]


For more details about hyperparameters, we refer to the software documentation of the `fmin2` in CMA-ES, which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API